<a href="https://colab.research.google.com/github/sekartalok/TFPICTURE.END/blob/main/trainingmulti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
import numpy as np
import tensorflow as tf

def load_and_prepare_data(npz_path='/content/drive/MyDrive/data_augmented.npz', batch_size=128):

    data = np.load(npz_path)

    train_images = data['train_images']
    train_labels = data['train_labels']
    val_images = data['val_images']
    val_labels = data['val_labels']


    train_images = train_images.astype(np.float32) / 255.0
    val_images = val_images.astype(np.float32) / 255.0


    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    validation_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))


    train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_dataset, validation_dataset




In [ ]:
train_dataset, validation_dataset=load_and_prepare_data()

In [2]:


def display_samples(dataset, num_samples=5):
    # Create an iterator from the dataset
    dataset_iterator = dataset.as_numpy_iterator()

    # Get the first batch of images and labels
    images, labels = next(dataset_iterator)

    # Check the shape of the images and labels
    print("Image batch shape:", images.shape)  # Debugging line to check shape
    print("Label batch shape:", labels.shape)  # Debugging line to check shape

    # Ensure you only take the first num_samples from the batch
    images = images[:num_samples]
    labels = labels[:num_samples]

    # Set up the plot
    plt.figure(figsize=(15, 5))

    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)  # Create a subplot for each sample
        plt.imshow(images[i])  # Display the image
        plt.title(f'Label: {np.argmax(labels[i])}')  # Display the label (assuming one-hot encoding)
        plt.axis('off')  # Hide axis

    plt.tight_layout()
    plt.show()

# Example usage to display training and validation samples
display_samples(train_dataset, num_samples=22)        # Display training samples



NameError: name 'train_dataset' is not defined

In [ ]:
SHUFFLE_BUFFER_SIZE = 128
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )


In [ ]:
# Define the model
def create_model(num_classes=9):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(200, 200, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(213, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
       # tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(510, activation='relu'),
        tf.keras.layers.Dense(125, activation='relu'),

        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
        metrics=['accuracy']
    )

    return model




In [ ]:
model = create_model()

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 198, 198, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 99, 99, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 97, 97, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 46, 46, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 21, 21, 213)         │         245,589 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 10, 10, 213)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 8, 8, 512)           │         982,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 510)                 │       4,178,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 125)                 │          63,875 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 125)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 9)                   │           1,134 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,564,292 (21.23 MB)

 Trainable params: 5,564,292 (21.23 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    # Define the correct function signature for on_epoch_end method
    def on_epoch_end(self, epoch, logs={}):

        if logs.get('accuracy', 0) >= 0.8 and logs.get('val_accuracy', 0) >= 0.8:
          self.model.stop_training = True

          print("\nReached 95% train accuracy and 80% validation accuracy, so cancelling training!")

In [ ]:
def load_image_from_npz(npz_path, index):
    """Loads an image from the specified .npz file."""

    data = np.load(npz_path)


    images = data['train_images']
    return images[index]

In [ ]:

def visualize_intermediate_layers(model, img_array, rescale_layer=None):
    """
    Visualizes the intermediate feature maps of a Keras model for a given input image array.

    Parameters:
    - model: Keras model to visualize.
    - img_array: Numpy array of the input image with shape (height, width, channels).
    - rescale_layer: Optional Keras layer to apply for rescaling the image.
    """

    # Define a new Model that will output intermediate representations
    successive_outputs = [layer.output for layer in model.layers[1:]]
    visualization_model = tf.keras.models.Model(inputs=model.inputs, outputs=successive_outputs)

    # Reshape the image to add the batch dimension
    x = img_array.reshape((1,) + img_array.shape)  # Shape should be (1, height, width, channels)

    # Optionally scale the image
    if rescale_layer is not None:
        x = rescale_layer(x)

    # Obtain intermediate representations for the input image
    successive_feature_maps = visualization_model.predict(x, verbose=False)

    # Get layer names for visualization
    layer_names = [layer.name for layer in model.layers[1:]]

    # Display the feature maps
    for layer_name, feature_map in zip(layer_names, successive_feature_maps):
        if len(feature_map.shape) == 4:
            n_features = feature_map.shape[-1]  # Number of features in the feature map
            size = feature_map.shape[1]  # Feature map size

            # Create a grid to tile the feature maps
            display_grid = np.zeros((size, size * n_features))
            for i in range(n_features):
                x = feature_map[0, :, :, i]
                x -= x.mean()
                x /= x.std() if x.std() > 0 else 1  # Avoid division by zero
                x *= 64
                x += 128
                x = np.clip(x, 0, 255).astype('uint8')

                # Tile each filter into the grid
                display_grid[:, i * size:(i + 1) * size] = x

            # Adjust the figure size for better visibility
            scale = 40. / n_features  # Increase scale to make images larger
            plt.figure(figsize=(scale * n_features, scale * 1.5))  # Adjust height for better visibility
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(display_grid, aspect='auto', cmap='viridis')
            plt.axis('off')  # Hide axis
            plt.show()




In [1]:
# Example Usage
npz_path = '/content/drive/MyDrive/data_augmented.npz'
image_index = 10  # Specify the index of the image you want to visualize

# Load the image from the .npz file
img = load_image_from_npz(npz_path, image_index)

# Ensure img is a 3D array (height, width, channels)
if img.ndim == 3:
    # Optionally define a rescale layer (if your model expects scaled inputs)
    rescale_layer = tf.keras.layers.Rescaling(1./255)  # Normalizing to [0, 1]

    # Call the visualization function
    visualize_intermediate_layers(model, img, rescale_layer=rescale_layer)
else:
    print("Error: Loaded image does not have the correct shape.")


NameError: name 'load_image_from_npz' is not defined

In [ ]:
import json

# Train the model and capture the training history
history = model.fit(
    train_dataset_final,
    epochs=129,
    validation_data=validation_dataset_final,
    callbacks=[EarlyStoppingCallback()]
)

# Save the trained model to an HDF5 file
model.save('my_model.h5')  # Save model

# Save the training history to a JSON file
with open('training_history.json', 'w') as f:
    json.dump(history.history, f)

# If you want to save history as a numpy array or CSV
history_data = np.array([history.history['loss'], history.history['val_loss'],
                          history.history['accuracy'], history.history['val_accuracy']]).T

np.savetxt('training_history.csv', history_data, delimiter=',',
           header='loss,val_loss,accuracy,val_accuracy', comments='')


Epoch 1/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 65s 12s/step - accuracy: 0.1273 - loss: 2.1952 - val_accuracy: 0.0833 - val_loss: 2.1947
Epoch 2/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 87s 13s/step - accuracy: 0.1787 - loss: 2.1761 - val_accuracy: 0.0833 - val_loss: 2.1703
Epoch 3/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 63s 14s/step - accuracy: 0.1900 - loss: 2.1489 - val_accuracy: 0.2361 - val_loss: 2.1114
Epoch 4/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 84s 13s/step - accuracy: 0.2127 - loss: 2.0927 - val_accuracy: 0.3194 - val_loss: 1.9984
Epoch 5/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 80s 14s/step - accuracy: 0.1988 - loss: 2.0042 - val_accuracy: 0.2639 - val_loss: 1.9228
Epoch 6/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 64s 14s/step - accuracy: 0.3078 - loss: 1.8776 - val_accuracy: 0.2917 - val_loss: 1.7981
Epoch 7/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 12s/step - accuracy: 0.2951 - loss: 1.8744 - val_accuracy: 0.3611 - val_loss: 1.7711
Epoch 8/129
5/5 ━━━━━━━━━━━━━━━━━━━━ 63s 14s/step - accuracy: 0.3399 - loss: 1.7508 - val_accuracy: 0.4444 - val_loss:

In [ ]:
# Get training and validation accuracies
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Training and validation accuracy')

for i, (data, label) in enumerate(zip([(acc, val_acc), (loss, val_loss)], ["Accuracy", "Loss"])):
    ax[i].plot(epochs, data[0], 'r', label="Training " + label)
    ax[i].plot(epochs, data[1], 'b', label="Validation " + label)
    ax[i].legend()
    ax[i].set_xlabel('epochs')

plt.show()

In [ ]:
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('/content/my_model.h5')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array

def load_and_predict(model, image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(200, 200))  # Resize to match model input
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = img_array.reshape((1,) + img_array.shape)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0, 1]

    # Make a prediction
    predictions = model.predict(img_array)

    # Print predictions and the class with the highest probability
    print("Predictions:", predictions)
    predicted_class = np.argmax(predictions[0])
    print("Predicted Class:", predicted_class)

    # Plot the image and prediction
    plt.imshow(img)
    plt.title(f"Predicted Class: {predicted_class}")
    plt.axis('off')
    plt.show()

# Example call
load_and_predict(model, '/content/Image_6(1).jpg')
